# 🏋️ Forma AI - Comprehensive Fitness Training

**IMPORTANT: Run cells ONE BY ONE, not "Run All"!**

This trains Forma AI - a bilingual fitness assistant with:
- Egyptian Arabic (العامية المصرية)
- Franco Arabic (3aml eh, ezayak)
- English

**Dataset:** 9,500+ samples including:
- Egyptian foods, fast food, supermarkets
- Workouts, calisthenics, yoga, flexibility
- Boxing, Muay Thai, BJJ, Kickboxing
- WHO guidelines, research-based advice
- Forma app integration & identity
- Equipment adaptations (gym/home)

**GPU:** L4 (recommended) - 3x faster than T4
**Time:** 3-4 hours for excellent quality

---

## Step 1: Check GPU (L4 recommended)

In [ ]:
# Check GPU - L4 is 3x faster than T4
!nvidia-smi --query-gpu=name,memory.total --format=csv

import torch
if not torch.cuda.is_available():
    print("❌ ERROR: No GPU!")
    print("Go to Runtime > Change runtime type > L4 GPU")
else:
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"✅ VRAM: {vram:.1f} GB")
    
    if "L4" in gpu_name:
        print("🚀 L4 detected! Fastest option for training.")
    elif "A100" in gpu_name:
        print("🚀 A100 detected! Premium GPU.")
    elif "T4" in gpu_name:
        print("⚠️ T4 detected. Works but L4 is 3x faster.")
    
    print("\n✅ Ready to train!")

## Step 2: Install Dependencies

**⚠️ After this cell, RESTART the session!**
- Go to Runtime > Restart session
- Then continue from Step 3

In [ ]:
# Install packages
!pip install -q transformers==4.46.0 datasets accelerate peft trl bitsandbytes
!pip install -q sentencepiece protobuf

print("")
print("=" * 60)
print("✅ INSTALLATION COMPLETE!")
print("=" * 60)
print("")
print("⚠️  NOW: Go to Runtime > Restart session")
print("⚠️  THEN: Continue from Step 3 (skip this cell)")
print("=" * 60)

## Step 3: Download Training Data

In [ ]:
import json
import urllib.request

# Download comprehensive training data
DATA_URL = "https://raw.githubusercontent.com/Abdellatifemara/Forma/main/apps/api/training-data/train_comprehensive.json"

print("📥 Downloading Forma AI training data...")
print("")
print("Dataset includes:")
print("  ✅ Egyptian foods, fast food, supermarkets")
print("  ✅ Workouts, calisthenics, yoga, flexibility")
print("  ✅ Boxing, Muay Thai, BJJ, Kickboxing (extensive)")
print("  ✅ WHO guidelines, Olympic training, research")
print("  ✅ Forma app identity & features")
print("  ✅ Equipment adaptations (gym/home)")
print("  ✅ Exercise FAQs & instructions")
print("  ✅ Goal-based recommendations")
print("  ✅ Egyptian mindset (assertive, motivating)")
print("")

urllib.request.urlretrieve(DATA_URL, "train_data.json")

with open("train_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"✅ Loaded {len(data):,} training samples!")

# Show category breakdown
categories = {}
for item in data:
    cat = item.get("category", "fitness")
    categories[cat] = categories.get(cat, 0) + 1

print("\nTop categories:")
for cat, count in sorted(categories.items(), key=lambda x: -x[1])[:15]:
    print(f"  {cat}: {count}")

## Step 4: Load Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Qwen2.5-3B - excellent Arabic support
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

print(f"📥 Loading {MODEL_NAME}...")
print("This takes 2-3 minutes...")

# 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Prepare for training
model = prepare_model_for_kbit_training(model)

# Add LoRA adapters - higher rank for better quality
lora_config = LoraConfig(
    r=64,              # Higher rank for better learning
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"")
print(f"✅ Model loaded!")
print(f"✅ Trainable parameters: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

## Step 5: Prepare Dataset

In [ ]:
from datasets import Dataset

# Forma AI System Prompt - This defines who Forma is
SYSTEM_PROMPT = """You are Forma AI - a bilingual fitness assistant designed for Egypt.

WHO YOU ARE:
- Your name is Forma (فورما)
- You are the AI inside the Forma fitness app
- You have an Egyptian mindset - assertive, motivating, direct
- You understand: Egyptian Arabic, Franco Arabic, and English

WHAT YOU KNOW:
- Exercise technique and programming (all types)
- Combat sports: Boxing, Muay Thai, BJJ, Kickboxing
- Nutrition with Egyptian foods, fast food, restaurants
- WHO guidelines and research-based recommendations
- Home workouts, resistance bands, bodyweight training
- Yoga, flexibility, and mixed training
- Supplements (evidence-based only)

HOW YOU HELP:
- You know the Forma app features and can guide users
- You adapt to user's equipment (gym vs home)
- You create workout and meal plans
- You track progress and give personalized advice
- You recommend doctors for injuries/medical issues

YOUR STYLE:
- Be direct and motivating
- Give practical, actionable advice
- Don't overcomplicate things
- Push users to be better, but be supportive"""

# Clean and prepare data
clean_data = []
for item in data:
    instruction = item.get('instruction', item.get('input', ''))
    output = item.get('output', item.get('response', ''))
    
    # Skip if either is empty/None
    if instruction and output and len(str(instruction)) > 0 and len(str(output)) > 0:
        clean_data.append({
            'instruction': str(instruction),
            'output': str(output),
            'category': item.get('category', 'fitness')
        })

print(f"Original samples: {len(data):,}")
print(f"Clean samples: {len(clean_data):,}")
print(f"Removed {len(data) - len(clean_data)} bad samples")

def format_chat(item):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": item['instruction']},
        {"role": "assistant", "content": item['output']},
    ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": text}

dataset = Dataset.from_list(clean_data)
dataset = dataset.map(format_chat)

print(f"")
print(f"✅ Dataset ready: {len(dataset):,} samples")
print(f"")
print("Example:")
print(dataset[100]['text'][:500] + "...")

## Step 6: TRAIN! 🚀

**Watch the loss value decrease:**
- Starting: ~2.5
- After 2000 steps: ~0.8
- After 5000 steps: ~0.4
- After 10000 steps: ~0.25 (excellent!)

**Training Time (L4 GPU):**
- 10,000 steps: ~3-4 hours
- Model saves every 2000 steps to Google Drive
- You can stop and resume if needed

In [ ]:
from trl import SFTTrainer, SFTConfig
from google.colab import drive
import time

# Mount Google Drive FIRST for checkpoints
drive.mount('/content/drive')
SAVE_PATH = "/content/drive/MyDrive/Forma-AI-Model"
!mkdir -p {SAVE_PATH}

# =====================================================
# TRAINING SETTINGS - Optimized for L4 GPU
# =====================================================
MAX_STEPS = 10000     # High quality - ~3-4 hours on L4
BATCH_SIZE = 4        # L4 can handle bigger batches
GRAD_ACCUM = 4        # Effective batch = 16
LEARNING_RATE = 1e-4  # Slower learning for better quality

print("=" * 60)
print("🚀 TRAINING STARTED")
print("=" * 60)
print(f"Steps: {MAX_STEPS:,}")
print(f"Effective batch size: {BATCH_SIZE * GRAD_ACCUM}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Estimated time on L4: ~3-4 hours")
print("")
print("📊 Watch the 'loss' value decrease over time!")
print("✅ Good final loss: < 0.3")
print("💾 Saves to Google Drive every 2000 steps")
print("=" * 60)

start_time = time.time()

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset,
    args=SFTConfig(
        output_dir=SAVE_PATH,  # Save directly to Drive
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,
        warmup_steps=200,
        max_steps=MAX_STEPS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=100,
        save_steps=2000,       # Save checkpoint every 2000 steps
        save_total_limit=3,    # Keep last 3 checkpoints
        optim="paged_adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=42,
        report_to="none",
        dataset_text_field="text",
        max_seq_length=2048,
    ),
)

# Train!
stats = trainer.train()

duration = (time.time() - start_time) / 60
print("")
print("=" * 60)
print("✅ TRAINING COMPLETE!")
print("=" * 60)
print(f"Time: {duration:.1f} minutes")
print(f"Final loss: {stats.training_loss:.4f}")
print("")
if stats.training_loss < 0.3:
    print("🏆 EXCELLENT! Loss is very low. Model learned exceptionally well.")
elif stats.training_loss < 0.5:
    print("✅ GOOD! Loss is low. Model learned well.")
elif stats.training_loss < 1.0:
    print("⚠️ OKAY. Model learned the basics. Consider more training.")
else:
    print("❌ HIGH. Something might be wrong. Check the logs.")

## Step 7: Save Final Model

In [ ]:
# Save the final model
print("💾 Saving final model to Google Drive...")

FINAL_PATH = "/content/drive/MyDrive/Forma-AI-Model/final"
!mkdir -p {FINAL_PATH}

model.save_pretrained(FINAL_PATH)
tokenizer.save_pretrained(FINAL_PATH)

print(f"✅ Model saved to: {FINAL_PATH}")

## Step 8: Test the Model

In [ ]:
# Test the trained model
model.eval()

test_prompts = [
    # English
    "Who are you?",
    "How many calories in Koshari?",
    "How do I do a proper bench press?",
    "What does WHO recommend for exercise?",
    "I want to learn boxing basics",
    # Arabic
    "انت مين؟",
    "كام سعرة في الفول؟",
    "ازاي ابني عضل؟",
    # Franco
    "kam calorie fel koshari?",
    "3ayz atamren fel beit men 8er gym",
]

print("🧪 Testing model...\n")
print("=" * 60)

for prompt in test_prompts:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract assistant response
    if "assistant" in response.lower():
        response = response.split("assistant")[-1].strip()
    
    print(f"Q: {prompt}")
    print(f"A: {response[:400]}")
    print("-" * 60)

## Step 9: Export to GGUF (for Ollama)

This converts the model to GGUF format for use with Ollama on your computer/server.

In [ ]:
# Merge LoRA weights with base model
print("🔄 Merging LoRA weights with base model...")
print("This takes 5-10 minutes...")

merged_model = model.merge_and_unload()

MERGED_PATH = "/content/forma-merged"
merged_model.save_pretrained(MERGED_PATH)
tokenizer.save_pretrained(MERGED_PATH)

print("✅ Model merged and saved!")

In [ ]:
# Convert to GGUF format
print("🔄 Converting to GGUF format...")
print("This takes 10-15 minutes...")

# Clone llama.cpp
!git clone --depth 1 https://github.com/ggerganov/llama.cpp
!pip install -q -r llama.cpp/requirements.txt

# Convert to GGUF (Q4_K_M is good balance of size/quality)
!python llama.cpp/convert_hf_to_gguf.py /content/forma-merged --outfile /content/forma-fitness.gguf --outtype q4_k_m

# Copy to Google Drive
!cp /content/forma-fitness.gguf "/content/drive/MyDrive/Forma-AI-Model/forma-fitness.gguf"

# Get file size
import os
size_gb = os.path.getsize("/content/forma-fitness.gguf") / (1024**3)

print("")
print("=" * 60)
print("✅ GGUF CONVERSION COMPLETE!")
print("=" * 60)
print("")
print(f"📦 File size: {size_gb:.2f} GB")
print("📁 Saved to: Google Drive > Forma-AI-Model > forma-fitness.gguf")
print("")
print("To use with Ollama:")
print("1. Download forma-fitness.gguf from Google Drive")
print("2. Create Modelfile with:")
print("   FROM ./forma-fitness.gguf")
print("   SYSTEM \"You are Forma AI...\"")
print("3. Run: ollama create forma-fitness -f Modelfile")
print("4. Test: ollama run forma-fitness")

## 🎉 Done!

Your Forma AI model is now:
1. **Saved in Google Drive** as LoRA adapter + tokenizer
2. **Exported as GGUF** for Ollama deployment

### What's Next?
1. Download the GGUF file from Google Drive
2. Set up Ollama on your server
3. Integrate with your Forma app API

### Model Capabilities:
- ✅ Egyptian Arabic understanding
- ✅ Franco Arabic support
- ✅ English fluency
- ✅ Egyptian foods & nutrition
- ✅ Workouts for gym and home
- ✅ Combat sports (Boxing, Muay Thai, BJJ, Kickboxing)
- ✅ WHO guidelines & research-based advice
- ✅ Forma app integration
- ✅ Assertive, motivating Egyptian style